<a href="https://colab.research.google.com/github/ntb-cicles/SMB-Linux-Windows/blob/main/Samba_com_a_controlador_de_domini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grup de treball VS Domini

Un **WORKGROUP** (grup de treball) és un **conjunt de màquines connectades a una mateixa xarxa**. En un workgroup **cada màquina gestiona de forma local els seus usuaris i contrasenyes**.

\

Un **DOMAIN** (domini) és també un **conjunt de màquines connectades a una mateixa xarxa**. Però en un domain, és **només una màquina, amb rol de servidor, qui gestiona els usuaris i contrasenyes de totes les màquines del domini de la xarxa**. 

\

Usualment els dominis s'utilitzen en xarxes amb un gran nombre d'equips, per exemple, en una empresa on hi han 60 ordinadors, és molt més fàcil gestionar tots els usuaris i contrasenyes dels 60 ordinadors des d'un servidor que pas haver d'anar un per un de forma local.

# Samba com a controlador de domini

Feu una nova còpia de seguretat del vostre fitxer de configuració de SAMBA (smb.conf) ja que tornarem a canviar-lo.

Comanda "hostnamectl" per saber el nom de la màquina que utilitzarem com a servidor.

In [ ]:
hostnamectl

In [ ]:
sudo apt update

In [ ]:
timedatectl

Kerberos es tracta d'un protocol que serveix per a autenticar dos dispositius que es connecten entre si. No significa que vagi a autoritzar-los, sinó autenticar-los. La seva funció és identificar cada usuari mitjançant xifratge.

In [ ]:
sudo apt-get install krb5-config

Kerberos us demana introduir 3 noms:

*   Kerberos realm: **EXAMPLE.COM**
*   Servidors de Kerberos que utilitzarem: **nodenil.example.com**
*   Servidor administratiu de Kerberos que utilitzarem: **nodenil.example.com**



Si us oblideu del nom que heu posat, es troba en el fitxer de configuració de kerberos, a la ruta **/etc/krb5.conf**

Necessitem winbind per unir manualment una instància de Linux amb un domini de Microsoft Active Directory

In [ ]:
sudo apt-get install winbind

Winbind us indica que vol canviar el vostre fitxer de configuració de SAMBA (smb.conf) per un altre nou. Accepteu-ho.

Si no el teniu instal·lat, també poseu el smbclient (el client de samba)

In [ ]:
sudo apt-get install smbclient

A mesura que la vostra màquina s'iniciï, haurà de conèixer l'assignació d'alguns noms d'amfitrió a adreces IP abans que es pugui fer referència al DNS. Aquesta assignació es manté al fitxer /etc/hosts. En absència d'un servidor de noms, qualsevol programa de xarxa del vostre sistema consulta aquest fitxer per determinar l'adreça IP que correspon a un nom d'amfitrió.

In [ ]:
nano /etc/hosts

In [ ]:
IPAddress       Hostname    		   Alias
127.0.0.1			  localhost	 	       deep.openna.com
208.164.186.1		deep.openna.com		 deep
208.164.186.2		mail.openna.com		 mail
208.164.186.3		web.openna.com		 web

Heu d'afegir una nova línia en aquest fitxers /etc/hosts, després de la resta de línies, amb la **IP de la vostra màquina** servidor, el **nom del servidor de Kerberos**, i un **alias que li vulgueu donar**. Per exemple:

In [ ]:
192.168.55.102		  nodenil.example.com	 	       nodenil

In [ ]:
cd /etc/samba
mv smb.conf smb.conf.install

In [ ]:
samba-tool domain provision

Deixeu tots els camps per defecte. Us hauria de sortir el nom de domini que heu posat anteriorment, DC (voleu que el servidor funcioni com a controlador de domini), i els DNS per defecte.

\

Només cal emplenar l'última opció, on cal posar una contrasenya per l'administrador del domini. Recordeu-la per més endavant!

Si no teniu la configuració de kerberos (krb5.conf) al directori /etc, cal moure-la:

In [ ]:
ls -l /etc/ | grep krb  # si surt, no cal fer la comanda següent!
cp /var/lib/samba/private/krb5.conf  /etc/

Cal desactivar els 4 serveis següents abans de continuar:

In [ ]:
systemctl disable --now smbd 
systemctl disable --now nmbd 
systemctl disable --now winbind 
systemctl disable --now systemd-resolved.service 
systemctl unmask samba-ad-dc.service

In [ ]:
systemctl enable --now samba-ad-dc.service

Comprovar que estem simulant un Active Directory de Windows amb el samba:

In [ ]:
samba-tool domain level show

Cal esborrar el link simbòlic següent, ja que hem de crear el nostre "resolv.conf" propi:

In [ ]:
rm /etc/resolv.conf
nano /etc/resolv.conf

Cal que hi poseu:

In [ ]:
nameserver 127.0.0.1